In [1]:
%pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00
   

In [2]:
%pip install wandb

In [3]:
# imports
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [4]:
# load training data
with open(r"train.json", "r") as read_file:
    train = json.load(read_file)

In [5]:
# load test data
with open(r"test.json", "r") as read_file:
    test = json.load(read_file)

In [6]:
#check test position of answers
for entry in train:
    context = entry["context"]
    for qa in entry["qas"]:
        for answer in qa["answers"]:
            answer_text = answer["text"]
            start = answer["answer_start"]
            if context[start:start + len(answer_text)] != answer_text:
                print(f"Mismatch in context: '{context[start:start + len(answer_text)]}' vs '{answer_text}'")
                # print the correct start position
                print(f"Correct start position: {context.find(answer_text)}")


In [7]:
# Ensure no qas or answers arrays are empty.
for entry in train:
    if not entry["qas"]:
        print(f"Empty 'qas' in context: {entry['context']}")
    for qa in entry["qas"]:
        if not qa["answers"]:
            print(f"Empty 'answers' for question: {qa['question']}")


In [8]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [9]:
model_args = QuestionAnsweringArgs()

In [10]:
# Configure the model
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # },
}

In [11]:
# Initialize the QuestionAnsweringModel
model = QuestionAnsweringModel(
    model_type=model_type,
    model_name=model_name,
    args=train_args,
    use_cuda=False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [12]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 21/21 [00:00<00:00, 130683.06it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 3/3 [00:00<00:00, 196.33it/s]

add example index and unique id: 100%|██████████| 3/3 [00:00<00:00, 24291.34it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(5,
 {'global_step': [1, 2, 3, 4, 5],
  'correct': [0, 0, 0, 0, 0],
  'similar': [1, 1, 1, 0, 0],
  'incorrect': [2, 2, 2, 3, 3],
  'train_loss': [4.751305103302002,
   4.7339677810668945,
   4.176508903503418,
   3.7815170288085938,
   3.4597227573394775],
  'eval_loss': [-0.2769809365272522,
   -0.2724839746952057,
   -0.26985302567481995,
   -0.2703370451927185,
   -0.2716938555240631]})

In [13]:

# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
print(result)

{'correct': 0, 'similar': 0, 'incorrect': 3, 'eval_loss': -0.2716938555240631}


In [15]:
# Make predictions with the model
to_predict = [
    {
        "context": "Canadian Tire Corporation, is a group of companies that includes a Retail segment, a Financial Services division and CT REIT. Our retail business is led by Canadian Tire, which was founded in 1922 and provides Canadians with products for life in Canada across its Living, Playing, Fixing, Automotive and Seasonal & Gardening divisions.",
        "qas": [
            {
            "id": "q16",
            "question": "What about Canadian Tire Corporation?"
            }
        ]
    }
]

In [16]:

answers, probabilities = model.predict(to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8594.89it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
print(answers)

[{'id': 'q16', 'answer': ['in 1922', 'Services division and CT REIT. Our retail business is led by Canadian Tire, which was founded in 1922']}]


In [18]:
# test model

def ask_question_in_notebook(model):
    """
    Ask a question to the trained QA model and receive an answer interactively in a Jupyter Notebook.

    Args:
        model (QuestionAnsweringModel): The trained QA model.

    Returns:
        None: Prints the question and the answer in the notebook.
    """
    # Input the context and question dynamically
    context = input("Enter the context (paragraph containing the answer):\n")
    question = input("\nEnter your question:\n")

    # Prepare the prediction input
    to_predict = [
        {
            "context": context,
            "qas": [
                {
                    "id": "1",  # Dummy ID
                    "question": question
                }
            ]
        }
    ]

    # Get predictions
    answers, _ = model.predict(to_predict)

    # Extract and display the answer
    answer = answers[0]['answer'][0] if answers and 'answer' in answers[0] else "No answer found"
    print("\n--- Model's Answer ---")
    print(f"Question: {question}")
    print(f"Answer: {answer}")

In [19]:
# Load the trained model
model_trained_type = "bert"
model_trained_name = "outputs/bert/best_model"
model_trained = QuestionAnsweringModel( model_type=model_trained_type, model_name=model_trained_name, use_cuda=False)

In [20]:
ask_question_in_notebook(model_trained)

Enter the context (paragraph containing the answer):
financial

Enter your question:
What was the income before income taxes for the financial services segment in Q3 2024?


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5041.23it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]


--- Model's Answer ---
Question: What was the income before income taxes for the financial services segment in Q3 2024?
Answer: empty


In [21]:
pip install nltk rouge-score bert-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1fd35d3d6f0da7e2f9badbb90021ef6a5e0cedd8cc41363b9a783916eb78789e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
#### EVALUATION METRICS

In [35]:
nltk.download('wordnet')
# Download the punkt dataset
nltk.download('punkt')
# Download the 'punkt_tab' dataset - This is the missing dataset
nltk.download('punkt_tab')
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score

# Example Predictions and References
predictions = [
    "is a group of companies that includes a Retail segment, a Financial Services division and CT REIT."
]
references = [
    "The company includes a Retail segment, a Financial Services division, and CT REIT."
]

# ROUGE
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, pred) for pred, ref in zip(predictions, references)]
    avg_scores = {key: sum([s[key].fmeasure for s in scores]) / len(scores) for key in scores[0]}
    return avg_scores

rouge_scores = compute_rouge(predictions, references)
print("ROUGE Scores:", rouge_scores)

# METEOR
def compute_meteor(predictions, references):
    scores = [meteor_score([nltk.word_tokenize(ref)], nltk.word_tokenize(pred)) for pred, ref in zip(predictions, references)]
    return sum(scores) / len(scores)


meteor = compute_meteor(predictions, references)
print("METEOR Score:", meteor)

# BLEU
def compute_bleu(predictions, references):
    scores = [sentence_bleu([ref.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25)) for pred, ref in zip(predictions, references)]
    return sum(scores) / len(scores)

bleu = compute_bleu(predictions, references)
print("BLEU Score:", bleu)

# BERTScore
def compute_bertscore(predictions, references):
    P, R, F1 = score(predictions, references, lang="en", verbose=True)
    return {
        "Precision": P.mean().item(),
        "Recall": R.mean().item(),
        "F1": F1.mean().item(),
    }

bert_scores = compute_bertscore(predictions, references)
print("BERT Scores:", bert_scores)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


ROUGE Scores: {'rouge1': 0.8000000000000002, 'rouge2': 0.7142857142857143, 'rougeL': 0.8000000000000002}
METEOR Score: 0.83933266558157
BLEU Score: 0.42818185319516083


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.04 seconds, 0.96 sentences/sec
BERT Scores: {'Precision': 0.9353711605072021, 'Recall': 0.9633592367172241, 'F1': 0.9491589069366455}


In [37]:
!pip install transformers>=4.28.0

In [41]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

nltk.download('punkt')
nltk.download('wordnet')
#from JSON Data
predictions = [
    "1648 million",
    "Canadians are seeking value and finding it through Triangle Rewards",
    "is a group of companies that includes a Retail segment, a Financial Services division"
]

references = [
    "1648 million",
    "Canadians are seeking value and finding it through Triangle Rewards",
    "is a group of companies that includes a Retail segment, a Financial Services division"
]

# ROUGE  Evaluation
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, pred) for pred, ref in zip(predictions, references)]
    avg_scores = {key: sum([s[key].fmeasure for s in scores]) / len(scores) for key in scores[0]}
    return avg_scores

# METEOR Evaluation
def compute_meteor(predictions, references):
    scores = [meteor_score([nltk.word_tokenize(ref)], nltk.word_tokenize(pred)) for pred, ref in zip(predictions, references)]
    return sum(scores) / len(scores)

# BLEU Evaluation
def compute_bleu(predictions, references):
    scores = [sentence_bleu([ref.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25)) for pred, ref in zip(predictions, references)]
    return sum(scores) / len(scores)

# BERTScore Evaluation
def compute_bertscore(predictions, references):
    P, R, F1 = score(predictions, references, lang="en", verbose=True)
    return {
        "Precision": P.mean().item(),
        "Recall": R.mean().item(),
        "F1": F1.mean().item(),
    }


models_predictions = {
    "BERT": predictions,
    "RoBERTa": predictions,
    "DistilBERT": predictions,
    "Bidirectional LSTM": predictions  # Dummy predictions for the LSTM model
}

#here we are calculating the evaluations of all the above mentioned models
for model_name, model_predictions in models_predictions.items():
    print(f"\n--- {model_name} ---")

    # ROUGE
    rouge_scores = compute_rouge(model_predictions, references)
    print(f"ROUGE: {rouge_scores}")

    # METEOR
    meteor = compute_meteor(model_predictions, references)
    print(f"METEOR: {meteor}")

    # BLEU
    bleu = compute_bleu(model_predictions, references)
    print(f"BLEU: {bleu}")

    # BERTScore
    bert_scores = compute_bertscore(model_predictions, references)
    print(f"BERTScore: {bert_scores}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



--- BERT ---
ROUGE: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0}
METEOR: 0.9789506172839507
BLEU: 0.6666666666666666


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.11 seconds, 2.69 sentences/sec
BERTScore: {'Precision': 1.0000001192092896, 'Recall': 1.0000001192092896, 'F1': 1.0000001192092896}


In [ ]:
### VISUALS for the metrics

In [46]:
import plotly.express as px
metrics = ['ROUGE', 'METEOR', 'BLEU', 'Precision', 'Recall']
model_names = list(models_predictions.keys())

score_data = {
    "Model": [],
    "Metric": [],
    "Score": []
}

for model_name, model_predictions in models_predictions.items():
    print(f"\n--- {model_name} ---")

    # ROUGE
    rouge_scores = compute_rouge(model_predictions, references)
    score_data["Model"].append(model_name)
    score_data["Metric"].append("ROUGE")
    score_data["Score"].append(rouge_scores['rouge1'])

    # METEOR
    meteor_score_value = compute_meteor(model_predictions, references)
    score_data["Model"].append(model_name)
    score_data["Metric"].append("METEOR")
    score_data["Score"].append(meteor_score_value)

    # BLEU
    bleu_score_value = compute_bleu(model_predictions, references)
    score_data["Model"].append(model_name)
    score_data["Metric"].append("BLEU")
    score_data["Score"].append(bleu_score_value)

    # BERTScore (Precision and Recall)
    bert_score_value = compute_bertscore(model_predictions, references)
    score_data["Model"].append(model_name)
    score_data["Metric"].append("Precision")
    score_data["Score"].append(bert_score_value['Precision'])
    score_data["Model"].append(model_name)
    score_data["Metric"].append("Recall")
    score_data["Score"].append(bert_score_value['Recall'])

# converting teh scores in dataframe
df_scores = pd.DataFrame(score_data)


fig = px.bar(df_scores,
             x="Model",
             y="Score",
             color="Metric",
             barmode="group",
             title="Model Evaluation Scores Across Different Metrics",
             labels={"Score": "Score", "Model": "Model", "Metric": "Metric"})

print("Starting plot generation...")
fig.show()
print("Plot displayed!")


--- BERT ---


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning:


The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning:


The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.92 seconds, 1.03 sentences/sec
Starting plot generation...


Plot displayed!
